In [15]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


In [16]:
choice = nn.Parameter(torch.empty(8, 768))
w1 = nn.Parameter(torch.empty(8, 768*4, 768))
w2 = nn.Parameter(torch.empty(8, 768, 768*4))
silu = nn.SiLU()
dropout = nn.Dropout(0.1)

nn.init.kaiming_uniform_(choice, a=math.sqrt(5))
nn.init.kaiming_uniform_(w1, a=math.sqrt(5))
nn.init.kaiming_uniform_(w2, a=math.sqrt(5))

x = torch.randn(3, 1024, 768)


In [17]:
choice = x @ torch.transpose(choice, -1, -2) # (batch_size, n_seq_len, n_experts)
choice = F.softmax(choice, dim=-1)


In [18]:
k = 1024 * 2 // 8 # 1024 * 2 // 8 = 256
G, I = torch.topk(torch.transpose(choice, -1, -2), k) # (batch_size, n_experts, k)
P = F.one_hot(I, num_classes=1024) # (batch_size, n_experts, k, n_seq_len)
P = P.to(x.dtype)
P.shape

torch.Size([3, 8, 256, 1024])

In [19]:
x_in = torch.einsum('beks,bsd->bekd', P, x) # (batch_size, n_experts, k, d_model)


In [20]:
x_mlp = silu(x_in @ torch.transpose(w1, -1, -2)) @ torch.transpose(w2, -1, -2)
x_mlp.shape


torch.Size([3, 8, 256, 768])

In [21]:
x_e = torch.einsum('beks,bekd->besd', P, x_mlp) # (batch_size, n_experts, k, d_model)
x_e.shape



torch.Size([3, 8, 1024, 768])

In [22]:
x_out = torch.einsum('beks,bek,besd->bsd', P, G, x_e)
x_out.shape


torch.Size([3, 1024, 768])

In [23]:
x_out = dropout(x_out)
x_out.shape

torch.Size([3, 1024, 768])